In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("logistic_regression").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [5]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_model = LogisticRegression()

In [7]:
fitted_model = my_model.fit(my_data)

In [9]:
log_summary = fitted_model.summary

In [11]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [12]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [13]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [14]:
final_model = LogisticRegression()

In [15]:
fit_final = final_model.fit(lr_train)

In [16]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [18]:
prediction_and_labels.accuracy

0.9487179487179487

In [20]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[33.3413311139116...|[0.99999999999999...|       0.0|
|  0.0|(692,[122,123,148...|[38.4861500001643...|[1.0,1.9305338690...|       0.0|
|  0.0|(692,[123,124,125...|[7.31367268715428...|[0.99933407854541...|       0.0|
|  0.0|(692,[124,125,126...|[24.9677723424378...|[0.99999999998565...|       0.0|
|  0.0|(692,[124,125,126...|[16.5464473592345...|[0.99999993484178...|       0.0|
|  0.0|(692,[124,125,126...|[27.0537602450566...|[0.99999999999821...|       0.0|
|  0.0|(692,[124,125,126...|[14.7934633883219...|[0.99999962391993...|       0.0|
|  0.0|(692,[125,126,127...|[26.0003919729671...|[0.99999999999489...|       0.0|
|  0.0|(692,[126,127,128...|[33.2974124820783...|[0.99999999999999...|       0.0|
|  0.0|(692,[126

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [22]:
my_evaluator = BinaryClassificationEvaluator()

In [23]:
my_final_roc = my_evaluator.evaluate(prediction_and_labels.predictions)

In [24]:
my_final_roc

0.9947368421052631